In [8]:
# BEFORE RUNNING: update paths in drumgan_evaluation/shell_scripts/fad.sh (lines 40 and 49)
# this notebook will run fad.sh to compute the frechet audio distance

import subprocess
from datetime import datetime
# from drumgan_evaluation.utils.utils import GPU_is_available
# from drumgan_evaluation.utils.utils import list_files_abs_path, list_files_in_subfolders, mkdir_in_path
import utils.utils as utils

In [ ]:
# configuration dictionary

# real samples used to train inception model
# "real_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/_footsteps_data/zapsplat_misc_shoes_misc_surfaces_inception_network",
# real samples used to train gan
# "real_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/_footsteps_data/zapsplat_pack_footsteps_high_heels_1s_aligned",
# gan synthesised samples
# "synth_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/ccwavegan-hifigan-fresh/checkpoints/2021-09-20_13h23m/120k_generated_audio_large",
# "synth_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/ccwavegan-hifigan-fresh/checkpoints/2021-09-20_19h46m/120k_generated_audio_large",

config = {
    "real_samples_path":    "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/_footsteps_data/zapsplat_misc_shoes_misc_surfaces_inception_network", # no end slash 
    "synth_samples_path":   "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/ccwavegan-hifigan-fresh/checkpoints/2021-09-20_19h46m/120k_generated_audio_large_1s",
    
    "output_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/ccwavegan-hifigan-fresh/drumgan_evaluation/evaluation",
    "output_folder": "evaluation_metrics",
}

In [ ]:
# list all real and synthesised samples in csv files
real_samples_path = config['real_samples_path']
synth_samples_path = config['synth_samples_path']

real_files = utils.list_files_in_subfolders(real_samples_path, format='.wav')
synth_files = utils.list_files_in_subfolders(synth_samples_path, format='.wav')

output_path = utils.mkdir_in_path(config['output_path'], config['output_folder'])
output_path = utils.mkdir_in_path(output_path, "fad")

real_paths_csv = f"{output_path}/real_audio.csv"
with open(real_paths_csv, "w") as f:
    for file_path in real_files:
        f.write(file_path + '\n')
synth_paths_csv = f"{output_path}/synth_audio.csv"
with open(synth_paths_csv, "w") as f:
    for file_path in synth_files:
        f.write(file_path + '\n')

In [ ]:
# run shell script to compute fad
# the script will - in turn - run the fad computation from the google_research_fad repo
fad = float(subprocess.check_output(["sh",
                        "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/ccwavegan-hifigan-fresh/drumgan_evaluation/shell_scripts/fad.sh",
                        "--real="+real_paths_csv,
                        "--synth="+synth_paths_csv,
                        "--output="+output_path]).decode()[-10:-1])

# save result
with open(f"{output_path}/fad_{len(synth_files)}_{datetime.now().strftime('%d-%m-%y_%H_%M')}.txt", "w") as f:
    f.write(str(fad))
    f.close()

print("FAD={0:.4f}".format(fad))